In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [5]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFace, HuggingFaceModel
import json

In [3]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [6]:
# Hub Model configuration. https://huggingface.co/models

trust_remote_code = True

hub = {
    'HF_MODEL_ID':'DAMO-NLP-SG/VideoLLaMA3-2B', # model_id from hf.co/models
    'HF_TASK':'visual-question-answering', # NLP task you want to use for predictions
    'HF_MODEL_TRUST_REMOTE_CODE': json.dumps(trust_remote_code)
}

In [7]:
huggingface_model = HuggingFaceModel(
   model_data="s3://YOUR_BUCKET/llava-full.tar.gz",
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.37", # transformers version used
   pytorch_version="2.1", # pytorch version used
   py_version="py310", # python version of the DLC
)

In [8]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g5.xlarge"
)

[03/09/25 05:47:25] INFO     Creating model with name:                                              ]8;id=812070;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=640930;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-03-09-05-47-25-489                                 

[03/09/25 05:47:26] INFO     Creating endpoint-config with name                                     ]8;id=241719;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=409096;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py#5937\5937]8;;\
                             huggingface-pytorch-inference-2025-03-09-05-47-26-130                                 

                    INFO     Creating endpoint with name                                            ]8;id=310782;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=421110;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py#4759\4759]8;;\
                             huggingface-pytorch-inference-2025-03-09-05-47-26-130                                 

-----------!

In [9]:
# example request, you always need to define "inputs"
image_path = "cat.png"
data = {
"inputs": {
    "content": [
            {
                "type": "image", 
                "image": {"image_path": image_path}
            },
            {
                "type": "text", 
                "text": "What is funny in the image?"},
        ]
    }
}

# request
predictor.predict(data)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:18                                                                                   │
│                                                                                                  │
│   15 }                                                                                           │
│   16                                                                                             │
│   17 # request                                                                                   │
│ ❱ 18 predictor.predict(data)                                                                     │
│   19                                                                                             │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/base_predictor.py: │
│ 212 in predict                                                                                   │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/botocore/client.py:570 in    │
│ _api_call                                                                                        │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/botocore/context.py:124 in   │
│ wrapper                                                                                          │
│                                                                                                  │
│   121 │   │   │   with start_as_current_context():                                               │
│   122 │   │   │   │   if hook:                                                                   │
│   123 │   │   │   │   │   hook()                                                                 │
│ ❱ 124 │   │   │   │   return func(*args, **kwargs)                                               │
│   125 │   │                                                                                      │
│   126 │   │   return wrapper                               